# PRODUCT DIMENSION

In [1]:
import pandas as pd
from connection_script import connect_databases

In [2]:
db_op, db_etl = connect_databases()

## EXTRACT 

In [3]:
dim_product_category =  pd.read_sql_query("SELECT * FROM [Production].[ProductCategory];", db_op)
dim_product_subcategory =  pd.read_sql_query("SELECT Name, ProductCategoryID FROM [Production].[ProductSubCategory];", db_op)
#dim_product =  pd.read_sql_query(
"""
    SELECT t1.*, t2.Name AS ModelName, t4.LargePhoto
    FROM [Production].[Product] AS t1
    INNER JOIN [Production].[ProductModel] AS t2
    ON t1.ProductModelID = t2.ProductModelID
    INNER JOIN [Production].[ProductProductPhoto] AS t3
    ON t1.ProductID = t3.ProductID
    INNER JOIN [Production].[ProductPhoto] AS t4
    ON t3.ProductPhotoID = t4.ProductPhotoID;
"""
#, db_op)
dim_product =  pd.read_sql_query("""
    SELECT t1.*, t2.Name AS ModelName, t4.LargePhoto
    FROM [Production].[Product] AS t1
    LEFT JOIN [Production].[ProductModel] AS t2
    ON t1.ProductModelID = t2.ProductModelID
    LEFT JOIN [Production].[ProductProductPhoto] AS t3
    ON t1.ProductID = t3.ProductID
    LEFT JOIN [Production].[ProductPhoto] AS t4
    ON t3.ProductPhotoID = t4.ProductPhotoID;
""", db_op)
product = pd.read_sql_query("SELECT * FROM [Production].[Product]", db_op)
print(f"Number of rows in dim_product: {dim_product.shape[0]}")
print(f"Number of rows in product: {product.shape[0]}")
dim_product

Number of rows in dim_product: 504
Number of rows in product: 504


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate,ModelName,LargePhoto
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,None,NaN,NaN,2008-04-30,NaT,None,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,None,NaN,NaN,2008-04-30,NaT,None,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,None,NaN,NaN,2008-04-30,NaT,None,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,None,NaN,NaN,2008-04-30,NaT,None,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,None,NaN,NaN,2008-04-30,NaT,None,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,796,"Road-250 Black, 58",BK-R89B-58,True,True,Black,100,75,1554.9479,2443.35,...,U,2.0,26.0,2012-05-30,NaT,None,1C530FE8-A169-4ADC-B3DC-B11A48245E63,2014-02-08 10:01:36.827,Road-250,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\xc6\xc8\x...
500,789,"Road-250 Red, 44",BK-R89R-44,True,True,Red,100,75,1518.7864,2443.35,...,U,2.0,26.0,2012-05-30,2013-05-29,None,0AA71AD6-AFAF-43C6-9745-35D815B50A5B,2014-02-08 10:01:36.827,Road-250,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x9fcj\xca...
501,790,"Road-250 Red, 48",BK-R89R-48,True,True,Red,100,75,1518.7864,2443.35,...,U,2.0,26.0,2012-05-30,2013-05-29,None,115DDADE-70E3-43F9-80DC-638DAEA271C4,2014-02-08 10:01:36.827,Road-250,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x9fcj\xca...
502,791,"Road-250 Red, 52",BK-R89R-52,True,True,Red,100,75,1518.7864,2443.35,...,U,2.0,26.0,2012-05-30,2013-05-29,None,C9FD1DF4-9512-420A-B379-067108033B75,2014-02-08 10:01:36.827,Road-250,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x9fcj\xca...


## TRANSFORM

### Product Category

In [4]:
dim_product_category.index+=1
dim_product_category = dim_product_category.rename(columns={"Name":"EnglishProductCategoryName"})
dim_product_category = dim_product_category.assign(ProductCategoryAlternateKey=list(dim_product_category.index))
dim_product_category = dim_product_category[["ProductCategoryAlternateKey", "EnglishProductCategoryName"]]
dim_product_category


,ProductCategoryAlternateKey,EnglishProductCategoryName
1,1,Bikes
2,2,Components
3,3,Clothing
4,4,Accessories


### Product Subcategory

In [5]:
dim_product_subcategory.index+=1
dim_product_subcategory = dim_product_subcategory.rename(columns={"Name":"EnglishProductSubcategoryName", "ProductCategoryID":"ProductCategoryKey"})
dim_product_subcategory = dim_product_subcategory.assign(ProductSubcategoryAlternateKey=list(dim_product_subcategory.index))
dim_product_subcategory = dim_product_subcategory[["ProductSubcategoryAlternateKey", "EnglishProductSubcategoryName", "ProductCategoryKey"]]
dim_product_subcategory

,ProductSubcategoryAlternateKey,EnglishProductSubcategoryName,ProductCategoryKey
1,1,Mountain Bikes,1
2,2,Road Bikes,1
3,3,Touring Bikes,1
4,4,Handlebars,2
5,5,Bottom Brackets,2
6,6,Brakes,2
7,7,Chains,2
8,8,Cranksets,2
9,9,Derailleurs,2
10,10,Forks,2


### Product

In [6]:
dim_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ProductID              504 non-null    int64         
 1   Name                   504 non-null    object        
 2   ProductNumber          504 non-null    object        
 3   MakeFlag               504 non-null    bool          
 4   FinishedGoodsFlag      504 non-null    bool          
 5   Color                  256 non-null    object        
 6   SafetyStockLevel       504 non-null    int64         
 7   ReorderPoint           504 non-null    int64         
 8   StandardCost           504 non-null    float64       
 9   ListPrice              504 non-null    float64       
 10  Size                   211 non-null    object        
 11  SizeUnitMeasureCode    176 non-null    object        
 12  WeightUnitMeasureCode  205 non-null    object        
 13  Weigh

In [7]:

dim_product = dim_product[["ProductID",
                            "ProductNumber",
                           "ProductSubcategoryID",
                           "WeightUnitMeasureCode",
                           "SizeUnitMeasureCode",
                           "Name",
                           "StandardCost",
                          "FinishedGoodsFlag",
                          "Color",
                           "SafetyStockLevel",
                           "ReorderPoint",
                           "ListPrice",
                           "Size",
                           ##"SizeRange" missing
                           "Weight",
                           "DaysToManufacture",
                           "ProductLine",
                           "Class",
                           "Style",
                           "ModelName",
                           "LargePhoto"
                          ]]


dim_product = dim_product.rename(columns={"ProductNumber":"ProductAlternateKey",
                                          "ProductSubcategoryID":"ProductSubcategoryKey",
                                          "Name":"EnglishProductName"
                                         })
  
                                        

In [8]:
products_descriptions = pd.read_sql_query("SELECT ProductID, TRIM(CultureID) AS CultureID, Description FROM [Production].[vProductAndDescription]", db_op)

In [9]:
products_descriptions

,ProductID,CultureID,Description
0,994,en,Chromoly steel.
1,995,en,Aluminum alloy cups; large diameter spindle.
2,996,en,Aluminum alloy cups and a hollow axle.
3,984,en,"Suitable for any type of riding, on or off-roa..."
4,985,en,"Suitable for any type of riding, on or off-roa..."
...,...,...,...
1759,894,ar,تصميم عريض الوصلات.
1760,894,fr,Conception liaison large.
1761,894,th,การออกแบบให้มีจุดเชื่อมกว้าง
1762,894,he,עיצוב רחב-חוליות.


In [10]:
# Get description for a product in a specif language
def get_description(product_id, language):
    row_data = products_descriptions[(products_descriptions['ProductID']==product_id)&(products_descriptions['CultureID']==language)]
    if len(row_data) > 0:
        return row_data.iloc[0]['Description']
    else:
        return None
    

In [11]:
#dim_product['EnglishDescription'] = dim_product[['ProductID']].apply(lambda x:int(x), index=0)
products_ids = list(dim_product['ProductID'])
english_descriptions = []
french_descriptions = []
chinese_descriptions = []
arabic_descriptions = []
hebrew_descriptions = []
thai_descriptions = []
for p in products_ids:
    english_descriptions.append(get_description(p, 'en'))
    french_descriptions.append(get_description(p, 'fr'))
    chinese_descriptions.append(get_description(p, 'zh-cht'))
    arabic_descriptions.append(get_description(p, 'ar'))
    hebrew_descriptions.append(get_description(p, 'he'))
    thai_descriptions.append(get_description(p, 'th'))
    ## German, Japanese and Turkish are still missing

In [12]:
dim_product= dim_product.assign(
    EnglishDescription=english_descriptions,
    FrenchDescription=french_descriptions,
    ChineseDescription=chinese_descriptions,
    ArabicDescriptions=arabic_descriptions,
    HebrewDescription=hebrew_descriptions,
    ThaiDescriptions=thai_descriptions
)

In [13]:
dim_product.sort_values(by='ProductID', inplace=True)
dim_product = dim_product.rename(columns={"ProductID":"ProductKey"})
dim_product

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,...,Class,Style,ModelName,LargePhoto,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescriptions,HebrewDescription,ThaiDescriptions
0,1,AR-5381,NaN,None,None,Adjustable Race,0.0000,False,None,1000,...,None,None,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,None,None,None,None,None,None
1,2,BA-8327,NaN,None,None,Bearing Ball,0.0000,False,None,1000,...,None,None,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,None,None,None,None,None,None
2,3,BE-2349,NaN,None,None,BB Ball Bearing,0.0000,False,None,800,...,None,None,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,None,None,None,None,None,None
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,0.0000,False,None,800,...,None,None,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,None,None,None,None,None,None
4,316,BL-2036,NaN,None,None,Blade,0.0000,False,None,800,...,None,None,None,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,995,BB-8107,5.0,G,None,ML Bottom Bracket,44.9506,True,None,500,...,M,None,ML Bottom Bracket,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,铝合金车圈；大直径脚蹬轴。,فناجين من سبيكة الألومنيوم؛ ذات محور دوران كبي...,כיסויים מסגסוגת אלומיניום; ציר רחב-קוטר.,ดุมอลูมิเนียมอัลลอยด์ แกนเพลาขนาดใหญ่
9,996,BB-9108,5.0,G,None,HL Bottom Bracket,53.9416,True,None,500,...,H,None,HL Bottom Bracket,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x00\x00\x...,Aluminum alloy cups and a hollow axle.,Cuvettes en alliage d'aluminium et axe creux.,铝合金车圈和空心轴。,فناجين من سبيكة الألومنيوم ومحور أجوف.,כיסויים מסגסוגת אלומיניום וציר חלול.,ดุมอลูมิเนียมอัลลอยด์และเพลากลวง
383,997,BK-R19B-44,2.0,LB,CM,"Road-750 Black, 44",343.6496,True,Black,100,...,L,U,Road-750,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x8avw\xf7...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...
384,998,BK-R19B-48,2.0,LB,CM,"Road-750 Black, 48",343.6496,True,Black,100,...,L,U,Road-750,b'GIF89a\xf0\x00\x95\x00\xf7\x00\x00\x8avw\xf7...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,入门级成人自行车；确保越野旅行或公路骑乘的舒适。快拆式车毂和轮缘。,إنها دراجة مناسبة للمبتدئين من البالغين؛ فهي ت...,"אופני מבוגרים למתחילים; מציעים רכיבה נוחה ""מחו...",จักรยานระดับเริ่มต้นสำหรับผู้ใหญ่ ให้ความสบายใ...


## LOAD

In [14]:
db_op, db_etl = connect_databases()
dim_product_category.to_sql('DimProductCategory', db_etl, if_exists='replace', index_label='ProductCategoryKey')
dim_product_subcategory.to_sql('DimProductSubcategory', db_etl, if_exists='replace', index_label='ProductSubcategoryKey')
#dim_product.set_index('ProductID', inplace=True)
dim_product.to_sql('DimProduct', db_etl, if_exists='replace', index=False)

24

In [15]:
test = pd.read_sql_query("SELECT * FROM DimProduct", db_etl)
test

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,StandardCost,FinishedGoodsFlag,Color,SafetyStockLevel,...,Class,Style,ModelName,LargePhoto,EnglishDescription,FrenchDescription,ChineseDescription,ArabicDescriptions,HebrewDescription,ThaiDescriptions
0,1,AR-5381,NaN,None,None,Adjustable Race,0.0000,False,None,1000,...,None,None,None,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,None,None,None,None,None,None
1,2,BA-8327,NaN,None,None,Bearing Ball,0.0000,False,None,1000,...,None,None,None,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,None,None,None,None,None,None
2,3,BE-2349,NaN,None,None,BB Ball Bearing,0.0000,False,None,800,...,None,None,None,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,None,None,None,None,None,None
3,4,BE-2908,NaN,None,None,Headset Ball Bearings,0.0000,False,None,800,...,None,None,None,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,None,None,None,None,None,None
4,316,BL-2036,NaN,None,None,Blade,0.0000,False,None,800,...,None,None,None,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,BB-8107,5.0,G,None,ML Bottom Bracket,44.9506,True,None,500,...,M,None,ML Bottom Bracket,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,Aluminum alloy cups; large diameter spindle.,Cuvettes en alliage d'aluminium ; axe de grand...,?????;???????,?????? ?? ????? ??????????? ??? ???? ????? ???...,??????? ??????? ?????????; ??? ???-????.,????????????????????? ???????????????
500,996,BB-9108,5.0,G,None,HL Bottom Bracket,53.9416,True,None,500,...,H,None,HL Bottom Bracket,GIF89að • ÷     €   € €€   €€ € €€€€€ÀÀÀÿ   ÿ ...,Aluminum alloy cups and a hollow axle.,Cuvettes en alliage d'aluminium et axe creux.,??????????,?????? ?? ????? ?????????? ????? ????.,??????? ??????? ????????? ???? ????.,????????????????????????????????
501,997,BK-R19B-44,2.0,LB,CM,"Road-750 Black, 44",343.6496,True,Black,100,...,L,U,Road-750,GIF89að • ÷  Švw÷÷÷íììÃÕ²hefãçÜôêêXTWåÜ×HEFnM...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,????????;????????????????????????,???? ????? ?????? ????????? ?? ????????? ??? ?...,"????? ??????? ????????; ?????? ????? ???? ""???...",????????????????????????????????? ????????????...
502,998,BK-R19B-48,2.0,LB,CM,"Road-750 Black, 48",343.6496,True,Black,100,...,L,U,Road-750,GIF89að • ÷  Švw÷÷÷íììÃÕ²hefãçÜôêêXTWåÜ×HEFnM...,Entry level adult bike; offers a comfortable r...,Vélo d'adulte d'entrée de gamme ; permet une c...,????????;????????????????????????,???? ????? ?????? ????????? ?? ????????? ??? ?...,"????? ??????? ????????; ?????? ????? ???? ""???...",????????????????????????????????? ????????????...
